# Product Research Project using Reviews Best Buy

**Context:** Vacuum cleaner is an essential home appliance of  a majority of the US households. As a result, customers are offered various options that come with different price tags. This project aims to analyze the pros and cons between a high-end vacuum and a cheap one in order to find out why and why not the customers would buy the products and whether it is worth it to pay more. This study will play as a research process in the Research and Development cycle which enables the optimization of both products and reduces post-launch problems. 
 
**Central questions:**
Which aspects of a luxurious vacuum cleaner outperform those of the cheaper one?
Which features of both chosen products receive the highest number of  positive/negative/neutral feedbacks?


---

## Part 1: Data acquisition

In this first part, we use `requests` and `Xpath` to mine data from the reviews page of 2 vacuum cleaners that are sold on `bestbuy.com`

In [1]:
#import neccessary Python libraries and packages
import os
import io
import sys
import importlib
import pandas as pd
from lxml import etree
import requests

htmlparser =  etree.HTMLParser() # Use a custom HTML parser to parse the response content into an XML Element

datadir = 'data'

During the mining process, we clarify `User-Agent` in order to avoid being identified as robot and being blocked from the website and store in `header` variable.

In [2]:
header = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.6.1 Safari/605.1.15"}

---

### 1. Mining process for Hoover Vacuum

**Data description:** Our first dataset, we retrieve customers reviews and relevant information from the reviews pages of Hoover Vacuum on `bestbuy.com`. The dataset has 6 columns represent 6 types of information we get from the reviews product pages: 
  - `UserID`: represents the unique ID of each customers that provided reviews for the product
  - `Time`: the time each review was posted
  - `Ratings` : ratings the customers gave for their experience with the products (out of 5)
  - `Helpful scores` : number of votes indicating that the review was helpful
  - `Unhelpful scores`: number of votes indicating that the review was unhelpful
  - `Reviews`: review text of each customer


**Data Mining Process:** 
We first retrieve data from the webpage using `requests.get`. We clarify our `User-Agent` using `headers` parameter in `request.get` call. Since there are multiple pages exhibiting customers reviews for Hoover vacuum, we use `for loop` to traverse through all the review pages. We store response for each page as `response1` and append them into the `pages1` list. On each review page, we use `xpath` to gather information needed and store the data in corresponding lists. 

In [3]:
# Traverse through HTML of multiple review pages and retreive page's root
pages1 = []
for i in range (1,3):
    url = "https://www.bestbuy.com/site/reviews/hoover-windtunnel-2-whole-home-rewind-cobalt-blue-iron-ore/6138362?variant=A&skuId=6138362&page={}"
    response1 = requests.get(url.format(i), headers = header)
    pages1.append(response1) #store all page's roots in a list
    assert response1.status_code == 200


# Create different list to store differnt types of information 
rating1 = []
times1 = []
helpful1 =[]
unhelpful1 = []
userid1 = []
comment1=[]
roots1 = []

# Loop through each root we have gained from above to mine the needed data
for page in pages1:
    tree1 = etree.parse(io.BytesIO(page.content), htmlparser)
    root1 = tree1.getroot()
    roots1 += root1
    path1 = '//*[@id="reviews-accordion"]/div[1]/ul'
    bb1 = root1.xpath(path1)[0]
    r1 = bb1.xpath('.//div[1]/p/text()')
    rating1 += r1
    t1 = bb1.xpath('.//div[@class="posted-date-ownership disclaimer v-m-right-xxs"]/time/@title')
    times1 += t1
    cmt1 = bb1.xpath('//div[@class="ugc-review-body"]/p/text()')
    comment1 += cmt1
    h1 = bb1.xpath('//div[@class="feedback-display"]/button[1]/text()')
    helpful1 += h1
    uh1 = bb1.xpath('//div[@class="feedback-display"]/button[2]/text()')
    unhelpful1 += uh1
    idA = bb1.xpath('//div[@class="ugc-author v-fw-medium body-copy-lg"]/strong/text()')
    id1 = [idA[i] for i in range(len(idA)) if i % 2 == 0]
    userid1 += id1

To store the data we have mined into an `.xml` file, we loop through all the roots we have in the `roots1` list, decode, and write them in the file using `.write`.

In [4]:
for root1 in roots1:
    bestbuy1 = etree.tostring(root1, pretty_print = True).decode('utf-8')
    with open(os.path.join(datadir, 'hoover.xml'), 'w') as f1:
        f1.write(bestbuy1)

We create `DoL1` to store opur information as a dictionary of list and create a `DataFrame` from that DoL.

In [5]:
for i in range(len(userid1)):
    userid1[i] = str(userid1[i])
    times1[i] = str(times1[i])
    helpful1[i] = str(helpful1[i])
    unhelpful1[i] = str(unhelpful1[i])
    comment1[i] = str(comment1[i])
    rating1[i] = str(rating1[i])

DoL1 = {'UserID': userid1, 'Time': times1, 'Reviews': comment1,
        'Ratings': rating1, 'Helpful_scores': helpful1, 'Unhelpful_scores': unhelpful1}
len(DoL1["UserID"])

40

In [6]:
Hoover = pd.DataFrame(DoL1)
Hoover["Product_name"] = "Hoover"
Hoover.head(22)

,UserID,Time,Reviews,Ratings,Helpful_scores,Unhelpful_scores,Product_name
0,QueenAudrey,"Aug 25, 2022 1:56 PM",Im very impressed ive had my vacuum for 2 week...,Rated 5 out of 5 stars,Helpful (3),Unhelpful (0),Hoover
1,JJTargaryan,"Dec 5, 2022 8:00 PM",Excellent vacuum. Easy to break down and clean...,Rated 5 out of 5 stars,Helpful (1),Unhelpful (0),Hoover
2,Guy1,"Sep 20, 2018 5:39 PM","Great vacuum, have owned it for almost 2 month...",Rated 4 out of 5 stars,Helpful (34),Unhelpful (3),Hoover
3,Valueminded,"Jan 18, 2020 12:24 PM",I bought this bagless Hoover on sale at Best B...,Rated 3 out of 5 stars,Helpful (40),Unhelpful (13),Hoover
4,bbcustomer,"Jul 5, 2020 8:06 AM","This is great, really sucks up a lot. Perfect...",Rated 5 out of 5 stars,Helpful (15),Unhelpful (1),Hoover
5,Blessed,"Jul 6, 2021 10:08 PM","Optimum and efficient,able to maneuver and sta...",Rated 5 out of 5 stars,Helpful (8),Unhelpful (1),Hoover
6,Vega,"Nov 15, 2022 6:26 PM",The picture shows the model with the brush rol...,Rated 2 out of 5 stars,Helpful (0),Unhelpful (0),Hoover
7,AZMom,"Oct 14, 2022 3:53 PM",I had an older Hoover Wind tunnel and I loved ...,Rated 1 out of 5 stars,Helpful (2),Unhelpful (1),Hoover
8,Fuji,"Mar 19, 2022 4:50 PM",Vacuums incredibly well but poorly constructed...,Rated 3 out of 5 stars,Helpful (2),Unhelpful (0),Hoover
9,Kevin17,"Sep 17, 2019 10:22 PM",This was the best investment I could make sinc...,Rated 5 out of 5 stars,Helpful (20),Unhelpful (8),Hoover


### 2. Mining process for Dyson Vacuum

**Data description:** Our first dataset, we retrieve customers reviews and relevant information from the reviews pages of Hoover Vacuum on `bestbuy.com`. The dataset has 6 columns represent 6 types of information we get from the reviews product pages: 
  - `UserID`: represents the unique ID of each customers that provided reviews for the product
  - `Time`: the time each review was posted
  - `Ratings` : ratings the customers gave for their experience with the products (out of 5)
  - `Helpful scores` : number of votes indicating that the review was helpful
  - `Unhelpful scores`: number of votes indicating that the review was unhelpful
  - `Reviews`: review text of each customer


**Data Mining Process:** 
We first retrieve data from the webpage using `requests.get`. We clarify our `User-Agent` using `headers` parameter in `request.get` call. Since there are multiple pages exhibiting customers reviews for Hoover vacuum, we use `for loop` to traverse through all the review pages. We store response for each page as `response1` and append them into the `pages1` list. On each review page, we use `xpath` to gather information needed and store the data in corresponding lists. 

In [7]:
# Traverse through HTML of multiple review pages and retreive page's root
pages2 = []
for x in range(1,3):
    url = "https://www.bestbuy.com/site/reviews/dyson-v15-detect-cordless-vacuum-yellow-nickel/6451330?variant=A&skuId=6451330&page={}"
    response2 = requests.get(url.format(x), headers = header)
    pages2.append(response2)
    assert response2.status_code == 200

# Create different list to store differnt types of information 
rating2 = []
times2 = []
helpful2 =[]
unhelpful2 = []
userid2 = []
comment2 = []
roots2 = []

# Loop through each root we have gained from above to mine the needed data
for page in pages2:
    tree2 = etree.parse(io.BytesIO(page.content), htmlparser)
    root2 = tree2.getroot()
    roots2 += root2
    path2 = '//*[@id="reviews-accordion"]/div[1]/ul'
    bb2 = root2.xpath(path2)[0]
    r2 = bb2.xpath('.//div[1]/p/text()')
    rating2 += r2
    t2 = bb2.xpath('.//div[@class="posted-date-ownership disclaimer v-m-right-xxs"]/time/@title')
    times2 += t2
    cmt2 = bb2.xpath('//div[@class="ugc-review-body"]/p/text()')
    comment2 += cmt2
    h2 = bb2.xpath('//div[@class="feedback-display"]/button[1]/text()')
    helpful2 += h2
    uh2 = bb2.xpath('//div[@class="feedback-display"]/button[2]/text()')
    unhelpful2 += uh2
    idB = bb2.xpath('//div[@class="ugc-author v-fw-medium body-copy-lg"]/strong/text()')
    id2 = [idB[i] for i in range(len(idB)) if i % 2 == 0]
    userid2 += id2

In [8]:
for root2 in roots2:
    bestbuy2 = etree.tostring(root2, pretty_print = True).decode('utf-8')
    with open(os.path.join(datadir, 'dyson.xml'), 'w') as f2:
        f2.write(bestbuy2)

In [9]:
for i in range(len(userid2)):
    userid2[i] = str(userid2[i])
    times2[i] = str(times2[i])
    helpful2[i] = str(helpful2[i])
    unhelpful2[i] = str(unhelpful2[i])
    comment2[i] = str(comment2[i])
    rating2[i] = str(rating2[i])

DoL2 = {'UserID': userid2, 'Time': times2, 'Reviews': comment2,
        'Ratings': rating2, 'Helpful_scores': helpful2, 'Unhelpful_scores': unhelpful2}

In [10]:
Dyson = pd.DataFrame(DoL2)
Dyson["Product_name"] = "Dyson"
Dyson.head(10)

,UserID,Time,Reviews,Ratings,Helpful_scores,Unhelpful_scores,Product_name
0,btd42,"Jun 22, 2021 10:59 AM",Background: \nThis is my first dyson. I am com...,Rated 5 out of 5 stars,Helpful (31),Unhelpful (5),Dyson
1,AGK2000,"Dec 6, 2022 9:58 AM",It is an expensive vacuum we got a discount on...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
2,LonG,"Dec 9, 2022 5:13 PM","I do laundry, dishes, you name the chore, I do...",Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
3,bestbuy,"Dec 3, 2022 12:17 PM",First and only battery powered vacuum I’ve eve...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
4,LPerera,"Dec 4, 2022 6:17 PM",Simply put this vacume is built so well and ca...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
5,Goose,"Dec 9, 2022 3:33 PM","Best vacuum every, very strong and db levels a...",Rated 5 out of 5 stars,Helpful (1),Unhelpful (0),Dyson
6,Karim,"Dec 2, 2022 4:41 PM",The best cordless dyson vacuum yet! From the s...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
7,Roxie,"Dec 8, 2022 5:10 PM",I love this vacuum but I will say it is heavie...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
8,User,"Nov 9, 2022 3:48 PM",Great for picking up dust an animal here. ￼The...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
9,Juanber44,"Dec 4, 2022 9:32 AM",Just what i was expecting when i saw it for fi...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson


**Data analysis plan:**
We plan to use ratings to generally assess the customers experience with the products. Other information such as `UserID`, `Time` will be used as independent variables to identify each unique review. In this project, to answer our central questions, we use our reviews to subjectively investigate wth aspects of the products receive the most positive/negative feedbacks using the Contextual Semantic Search (CSS) library. 

In [11]:
Dyson.head(26)

,UserID,Time,Reviews,Ratings,Helpful_scores,Unhelpful_scores,Product_name
0,btd42,"Jun 22, 2021 10:59 AM",Background: \nThis is my first dyson. I am com...,Rated 5 out of 5 stars,Helpful (31),Unhelpful (5),Dyson
1,AGK2000,"Dec 6, 2022 9:58 AM",It is an expensive vacuum we got a discount on...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
2,LonG,"Dec 9, 2022 5:13 PM","I do laundry, dishes, you name the chore, I do...",Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
3,bestbuy,"Dec 3, 2022 12:17 PM",First and only battery powered vacuum I’ve eve...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
4,LPerera,"Dec 4, 2022 6:17 PM",Simply put this vacume is built so well and ca...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
5,Goose,"Dec 9, 2022 3:33 PM","Best vacuum every, very strong and db levels a...",Rated 5 out of 5 stars,Helpful (1),Unhelpful (0),Dyson
6,Karim,"Dec 2, 2022 4:41 PM",The best cordless dyson vacuum yet! From the s...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
7,Roxie,"Dec 8, 2022 5:10 PM",I love this vacuum but I will say it is heavie...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
8,User,"Nov 9, 2022 3:48 PM",Great for picking up dust an animal here. ￼The...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson
9,Juanber44,"Dec 4, 2022 9:32 AM",Just what i was expecting when i saw it for fi...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson


---

**Data citation:**

Bestbuy.com. Dyson - V15 Detect Cordless Vacuum - Yellow/Nickel. https://www.bestbuy.com/site/dyson-v15-detect-cordless-vacuum-yellow-nickel/6451330.p?skuId=6451330

Bestbuy.com. Hoover WindTunnel 2 Whole Home Rewind - Cobalt Blue & Iron Ore. https://www.bestbuy.com/site/reviews/hoover-windtunnel-2-whole-home-rewind-cobalt-blue-iron-ore/6138362?variant=A&skuId=6138362



## Part 2: Data storage

Break down `Time` column into date, time, year.
Calculate positivity scores 
Tokenize reviews and calculate `Similarity` scores. 

**Note** Having to use arg `ignore_index`

In [12]:
#reviews = pd.concat([Dyson, Hoover], axis=0, keys=pd.Index(['Dyson', 'Hoover'], name="Product Name"))
reviews = pd.concat([Dyson, Hoover], axis=0, ignore_index=True, keys=pd.Index(['Dyson', 'Hoover'], name="Product_name"))

#reviews.reset_index(level = 'Product Name', inplace = True)

In [13]:
reviews.tail()

,UserID,Time,Reviews,Ratings,Helpful_scores,Unhelpful_scores,Product_name
75,HenryR,"Dec 2, 2022 4:42 PM",Seems to work just fine have not used all the ...,Rated 4 out of 5 stars,Helpful (0),Unhelpful (0),Hoover
76,NJGecko,"Dec 18, 2019 8:43 AM",I bought this for a small apartment. I use it...,Rated 5 out of 5 stars,Helpful (19),Unhelpful (0),Hoover
77,Smokie,"Oct 28, 2022 9:25 PM","Very pleased with the vacuum, It replaced my ...",Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Hoover
78,Anonymous,"Nov 13, 2022 7:35 AM",I was tired of renting vacuum. I need it all i...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Hoover
79,Odavyn,"Sep 2, 2022 3:56 PM",You can never go wrong with a Hoover! Our old ...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Hoover


Use `.concat` then `reset_index` messed up the order of the data frame.

### A. Data cleaning

#### 1. Break down `Time` column:

In [14]:
time = []
date = []
year = []
month = []

for val0 in reviews["Time"]:
    time.append(val0[-8:])
    val0=val0[:-8]
    year.append(int(val0[-5:]))
    val0 = val0[:-5]
    date.append(val0[3:-1])
    month.append(val0[:3])
date1 = []
for val1 in date:
    if val1[-1]==',':
        date1.append(int(val1[:-1]))
    else:
        date1.append(int(val1))

reviews["Exact_time"] = time
reviews["Date"] = date1
reviews["Year"] = year
reviews["Month"] = month
#reviews.pop("Time")
        
reviews.head(10)

,UserID,Time,Reviews,Ratings,Helpful_scores,Unhelpful_scores,Product_name,Exact_time,Date,Year,Month
0,btd42,"Jun 22, 2021 10:59 AM",Background: \nThis is my first dyson. I am com...,Rated 5 out of 5 stars,Helpful (31),Unhelpful (5),Dyson,10:59 AM,22,2021,Jun
1,AGK2000,"Dec 6, 2022 9:58 AM",It is an expensive vacuum we got a discount on...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson,9:58 AM,6,2022,Dec
2,LonG,"Dec 9, 2022 5:13 PM","I do laundry, dishes, you name the chore, I do...",Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson,5:13 PM,9,2022,Dec
3,bestbuy,"Dec 3, 2022 12:17 PM",First and only battery powered vacuum I’ve eve...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson,12:17 PM,3,2022,Dec
4,LPerera,"Dec 4, 2022 6:17 PM",Simply put this vacume is built so well and ca...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson,6:17 PM,4,2022,Dec
5,Goose,"Dec 9, 2022 3:33 PM","Best vacuum every, very strong and db levels a...",Rated 5 out of 5 stars,Helpful (1),Unhelpful (0),Dyson,3:33 PM,9,2022,Dec
6,Karim,"Dec 2, 2022 4:41 PM",The best cordless dyson vacuum yet! From the s...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson,4:41 PM,2,2022,Dec
7,Roxie,"Dec 8, 2022 5:10 PM",I love this vacuum but I will say it is heavie...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson,5:10 PM,8,2022,Dec
8,User,"Nov 9, 2022 3:48 PM",Great for picking up dust an animal here. ￼The...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson,3:48 PM,9,2022,Nov
9,Juanber44,"Dec 4, 2022 9:32 AM",Just what i was expecting when i saw it for fi...,Rated 5 out of 5 stars,Helpful (0),Unhelpful (0),Dyson,9:32 AM,4,2022,Dec


#### 2. Edit `Ratings` column:

In [15]:
rating_edited = []
for val2 in reviews['Ratings']:
    rating_edited.append(int(val2[6]))
reviews['Ratings'] = rating_edited

In [16]:
reviews.head(10)

,UserID,Time,Reviews,Ratings,Helpful_scores,Unhelpful_scores,Product_name,Exact_time,Date,Year,Month
0,btd42,"Jun 22, 2021 10:59 AM",Background: \nThis is my first dyson. I am com...,5,Helpful (31),Unhelpful (5),Dyson,10:59 AM,22,2021,Jun
1,AGK2000,"Dec 6, 2022 9:58 AM",It is an expensive vacuum we got a discount on...,5,Helpful (0),Unhelpful (0),Dyson,9:58 AM,6,2022,Dec
2,LonG,"Dec 9, 2022 5:13 PM","I do laundry, dishes, you name the chore, I do...",5,Helpful (0),Unhelpful (0),Dyson,5:13 PM,9,2022,Dec
3,bestbuy,"Dec 3, 2022 12:17 PM",First and only battery powered vacuum I’ve eve...,5,Helpful (0),Unhelpful (0),Dyson,12:17 PM,3,2022,Dec
4,LPerera,"Dec 4, 2022 6:17 PM",Simply put this vacume is built so well and ca...,5,Helpful (0),Unhelpful (0),Dyson,6:17 PM,4,2022,Dec
5,Goose,"Dec 9, 2022 3:33 PM","Best vacuum every, very strong and db levels a...",5,Helpful (1),Unhelpful (0),Dyson,3:33 PM,9,2022,Dec
6,Karim,"Dec 2, 2022 4:41 PM",The best cordless dyson vacuum yet! From the s...,5,Helpful (0),Unhelpful (0),Dyson,4:41 PM,2,2022,Dec
7,Roxie,"Dec 8, 2022 5:10 PM",I love this vacuum but I will say it is heavie...,5,Helpful (0),Unhelpful (0),Dyson,5:10 PM,8,2022,Dec
8,User,"Nov 9, 2022 3:48 PM",Great for picking up dust an animal here. ￼The...,5,Helpful (0),Unhelpful (0),Dyson,3:48 PM,9,2022,Nov
9,Juanber44,"Dec 4, 2022 9:32 AM",Just what i was expecting when i saw it for fi...,5,Helpful (0),Unhelpful (0),Dyson,9:32 AM,4,2022,Dec


#### 2. Edit `Helpful scores` and `Unhelpful scores` column:

In [17]:
hf_edited = []
uhf_edited = []
for val3, val4 in zip(reviews['Helpful_scores'], reviews['Unhelpful_scores']):
    hf_edited.append(int(val3[9:-1]))
    uhf_edited.append(int(val4[11:-1]))
    

In [18]:
reviews['Helpful_scores'] = hf_edited
reviews['Unhelpful_scores'] = uhf_edited
reviews.head(10)

,UserID,Time,Reviews,Ratings,Helpful_scores,Unhelpful_scores,Product_name,Exact_time,Date,Year,Month
0,btd42,"Jun 22, 2021 10:59 AM",Background: \nThis is my first dyson. I am com...,5,31,5,Dyson,10:59 AM,22,2021,Jun
1,AGK2000,"Dec 6, 2022 9:58 AM",It is an expensive vacuum we got a discount on...,5,0,0,Dyson,9:58 AM,6,2022,Dec
2,LonG,"Dec 9, 2022 5:13 PM","I do laundry, dishes, you name the chore, I do...",5,0,0,Dyson,5:13 PM,9,2022,Dec
3,bestbuy,"Dec 3, 2022 12:17 PM",First and only battery powered vacuum I’ve eve...,5,0,0,Dyson,12:17 PM,3,2022,Dec
4,LPerera,"Dec 4, 2022 6:17 PM",Simply put this vacume is built so well and ca...,5,0,0,Dyson,6:17 PM,4,2022,Dec
5,Goose,"Dec 9, 2022 3:33 PM","Best vacuum every, very strong and db levels a...",5,1,0,Dyson,3:33 PM,9,2022,Dec
6,Karim,"Dec 2, 2022 4:41 PM",The best cordless dyson vacuum yet! From the s...,5,0,0,Dyson,4:41 PM,2,2022,Dec
7,Roxie,"Dec 8, 2022 5:10 PM",I love this vacuum but I will say it is heavie...,5,0,0,Dyson,5:10 PM,8,2022,Dec
8,User,"Nov 9, 2022 3:48 PM",Great for picking up dust an animal here. ￼The...,5,0,0,Dyson,3:48 PM,9,2022,Nov
9,Juanber44,"Dec 4, 2022 9:32 AM",Just what i was expecting when i saw it for fi...,5,0,0,Dyson,9:32 AM,4,2022,Dec


### B. Calculate sentiment scores

In [19]:
import spacy

install `spacy`: `pip install -U spacy`

In [20]:
nlp = spacy.load("en_core_web_lg")

Documentation: https://spacy.io/usage/linguistic-features

Tokenize and filter out nouns in reviews.

In [21]:
#feature = nlp('suction')
noun = []
#for i in range(len(reviews["Reviews"])):
for i in range(len(reviews["Reviews"])):
    reviews["Reviews"][i] = str(reviews["Reviews"][i])

for e in range(len(reviews["Reviews"])):
    noun_in_line = [token.lemma_ for token in nlp(reviews["Reviews"][e]) if token.pos_ == 'NOUN']
    noun.append(noun_in_line)

print(len(noun))

/var/folders/1h/g7_t3cm96633vmg94jxztth80000gn/T/ipykernel_90605/933517206.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews["Reviews"][i] = str(reviews["Reviews"][i])


80


Calculate similarity

In [22]:
def cal_similarity(feature):
    feature = nlp(feature)
    similarity_scores = []
    for l in range(len(noun)):
        tk = [nlp(token) for token in noun[l]]
        if len(tk) <= 0:
            similarity_scores.append(0)
        else:
            tk_similarity = [val.similarity(feature) for val in tk]
            similarity_scores.append(sum(tk_similarity)/len(tk_similarity))
    return similarity_scores

reviews['Suction_similairity_score'] = cal_similarity('su')
reviews['Price_similarity_score'] = cal_similarity('price')

reviews.tail(10)
    #print(noun[l][c for c in noun[l]].similarity(feature))

/var/folders/1h/g7_t3cm96633vmg94jxztth80000gn/T/ipykernel_90605/2861003487.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  tk_similarity = [val.similarity(feature) for val in tk]
/var/folders/1h/g7_t3cm96633vmg94jxztth80000gn/T/ipykernel_90605/2861003487.py:9: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  tk_similarity = [val.similarity(feature) for val in tk]


,UserID,Time,Reviews,Ratings,Helpful_scores,Unhelpful_scores,Product_name,Exact_time,Date,Year,Month,Suction_similairity_score,Price_similarity_score
70,raeddy77,"Jan 14, 2020 12:32 PM","We have been using a cordless, battery powered...",5,21,12,Hoover,12:32 PM,14,2020,Jan,0.028339,0.223866
71,Buddy,"Dec 14, 2022 5:17 PM",Surprisingly this vacuum really does the trick...,5,0,0,Hoover,5:17 PM,14,2022,Dec,0.026009,0.158290
72,Brett,"Apr 15, 2021 5:18 AM",Great Vacuum. A lot of power and gets the job ...,4,6,0,Hoover,5:18 AM,15,2021,Apr,0.052049,0.203668
73,TheDude,"Jan 1, 2022 9:46 AM",This is our second Hoover. The last one all t...,4,1,0,Hoover,9:46 AM,1,2022,Jan,0.015447,0.234062
74,Cherry,"Oct 22, 2022 5:26 PM","It's s good vacuum, as far as, getting the job...",4,0,0,Hoover,5:26 PM,22,2022,Oct,0.015281,0.196007
75,HenryR,"Dec 2, 2022 4:42 PM",Seems to work just fine have not used all the ...,4,0,0,Hoover,4:42 PM,2,2022,Dec,0.014404,0.184298
76,NJGecko,"Dec 18, 2019 8:43 AM",I bought this for a small apartment. I use it...,5,19,0,Hoover,8:43 AM,18,2019,Dec,0.030778,0.187824
77,Smokie,"Oct 28, 2022 9:25 PM","Very pleased with the vacuum, It replaced my ...",5,0,0,Hoover,9:25 PM,28,2022,Oct,-0.000097,0.224110
78,Anonymous,"Nov 13, 2022 7:35 AM",I was tired of renting vacuum. I need it all i...,5,0,0,Hoover,7:35 AM,13,2022,Nov,0.032089,0.217561
79,Odavyn,"Sep 2, 2022 3:56 PM",You can never go wrong with a Hoover! Our old ...,5,0,0,Hoover,3:56 PM,2,2022,Sep,0.027473,0.296698


Calculate sentimental scores for the reviews

In [23]:
import nltk
#nltk.download()

In [24]:
from nltk.sentiment import SentimentIntensityAnalyzer #import from nltk.sentiment module the SentimentIntensityAnalyzer

#create a SentimentIntensityAnalyzer object to calculate review sentiment intensity as sia
sia = SentimentIntensityAnalyzer() 

In [25]:
neg_score = []
pos_score = []
neu_score = []
for n in range(len(reviews["Reviews"])):
    sent_score = sia.polarity_scores(reviews["Reviews"][n])
    neg_score.append(sent_score['neg'])
    neu_score.append(sent_score['neu'])
    pos_score.append(sent_score['pos'])

reviews["Negative_rate"] = neg_score
reviews["Positive_rate"] = pos_score
reviews["Neutral_rate"] = neu_score
reviews.head(10)

print(len(reviews))

80


Create `review_feature` table

In [26]:
threshold1 = reviews['Suction_similairity_score'].mean()
threshold2 = reviews['Price_similarity_score'].mean()

In [27]:
features_mentioned = []
for i in range(len(reviews["Reviews"])):
    f = []
    if reviews["Suction_similairity_score"][i] >= threshold1:
        d = {"Product":reviews["Product_name"][i] ,"Reviews": reviews["Reviews"][i], "Feature": "Suction",\
            "Positive_rate": reviews['Positive_rate'][i], "Negative_rate": reviews["Negative_rate"][i], "Neutral_rate": reviews["Neutral_rate"][i]}
        features_mentioned.append(d)
    if reviews["Price_similarity_score"][i] >= threshold2:
        d = {"Product":reviews["Product_name"][i],"Reviews": reviews["Reviews"][i], "Feature": "Price",\
            "Positive_rate": reviews['Positive_rate'][i], "Negative_rate": reviews["Negative_rate"][i], "Neutral_rate": reviews["Neutral_rate"][i]}
        features_mentioned.append(d)
#    else:
#        pass
    
review_feature = pd.DataFrame(features_mentioned, columns=["Product", "Reviews", "Feature", "Positive_rate", "Negative_rate", "Neutral_rate"])
review_feature.head(22)

,Product,Reviews,Feature,Positive_rate,Negative_rate,Neutral_rate
0,Dyson,Background: \nThis is my first dyson. I am com...,Suction,0.242,0.047,0.711
1,Dyson,Background: \nThis is my first dyson. I am com...,Price,0.242,0.047,0.711
2,Dyson,It is an expensive vacuum we got a discount on...,Price,0.106,0.110,0.784
3,Dyson,"I do laundry, dishes, you name the chore, I do...",Suction,0.164,0.071,0.766
4,Dyson,First and only battery powered vacuum I’ve eve...,Suction,0.143,0.000,0.857
5,Dyson,"Best vacuum every, very strong and db levels a...",Suction,0.373,0.000,0.627
6,Dyson,"Best vacuum every, very strong and db levels a...",Price,0.373,0.000,0.627
7,Dyson,The best cordless dyson vacuum yet! From the s...,Price,0.399,0.000,0.601
8,Dyson,Just what i was expecting when i saw it for fi...,Price,0.219,0.000,0.781
9,Dyson,I had the first cordless and it could barely r...,Suction,0.128,0.059,0.813


In [28]:
review_user = reviews.loc[:, ("UserID", "Reviews", "Exact_time", "Date", "Month", "Year")]

In [29]:
review_rating = reviews.loc[:, ("UserID", "Reviews", "Ratings")]
review_rating.head()

,UserID,Reviews,Ratings
0,btd42,Background: \nThis is my first dyson. I am com...,5
1,AGK2000,It is an expensive vacuum we got a discount on...,5
2,LonG,"I do laundry, dishes, you name the chore, I do...",5
3,bestbuy,First and only battery powered vacuum I’ve eve...,5
4,LPerera,Simply put this vacume is built so well and ca...,5


---

## Part 3: Create SQL tables

**Create connection**

In [30]:
import pandas as pd
import os
import os.path
import json
import sqlalchemy as sa

module_dir = "../modules"
module_path = os.path.abspath(module_dir)
if not module_path in sys.path:
    sys.path.append(module_path)

%reload_ext sql

In [31]:
def getsqlite_creds(dirname=".",filename="creds.json",source="sqlite"):
    """ Using directory and filename parameters, open a credentials file
        and obtain the two parts needed for a connection string to
        a local provider using the "sqlite" dictionary within
        an outer dictionary.  
        
        Return a scheme and a dbfile
    """
    assert os.path.isfile(os.path.join(dirname, filename))
    with open(os.path.join(dirname, filename)) as f:
        D = json.load(f)
    sqlite = D[source]
    return sqlite["scheme"], sqlite["dbdir"], sqlite["database"]

In [32]:
def buildConnectionString(source="sqlite"):
    scheme, dbdir, database = getsqlite_creds(source=source)
    template = '{}:///{}/{}.db'
    return template.format(scheme, dbdir, database)

In [33]:
# Build the conection string
cstring = buildConnectionString("sqlite")
print("Connection string:", cstring)

# Connect to the database
engine = sa.create_engine(cstring)
connection = engine.connect()

Connection string: sqlite:///./reviews.db


**Create `review_user` table**

In [34]:
statement1 = """
CREATE TABLE IF NOT EXISTS review_user (
    UserID VARCHAR(100) NOT NULL,
    Reviews VARCHAR(100000) NOT NULL,
    Exact_time VARCHAR(100),
    Date VARCHAR(100),
    Month VARCHAR(5),
    Year INT,
    PRIMARY KEY (UserID, Exact_time, Date, Month)
)
"""
try:
    connection.execute(statement1)
except sa.exc.SQLAlchemyError as err:
    print("CREATE of review_user failed:", str(err))

In [35]:
review_user.to_sql("review_user", con=connection, if_exists="append", index=False)

80

In [36]:
df3 = pd.read_sql_table("review_user", connection)
df3

,UserID,Reviews,Exact_time,Date,Month,Year
0,btd42,Background: \nThis is my first dyson. I am com...,10:59 AM,22,Jun,2021
1,AGK2000,It is an expensive vacuum we got a discount on...,9:58 AM,6,Dec,2022
2,LonG,"I do laundry, dishes, you name the chore, I do...",5:13 PM,9,Dec,2022
3,bestbuy,First and only battery powered vacuum I’ve eve...,12:17 PM,3,Dec,2022
4,LPerera,Simply put this vacume is built so well and ca...,6:17 PM,4,Dec,2022
...,...,...,...,...,...,...
75,HenryR,Seems to work just fine have not used all the ...,4:42 PM,2,Dec,2022
76,NJGecko,I bought this for a small apartment. I use it...,8:43 AM,18,Dec,2019
77,Smokie,"Very pleased with the vacuum, It replaced my ...",9:25 PM,28,Oct,2022
78,Anonymous,I was tired of renting vacuum. I need it all i...,7:35 AM,13,Nov,2022


**Create `review_feature` table**

In [37]:
statement2 = """
CREATE TABLE IF NOT EXISTS review_feature (
    Product VARCHAR(50) NOT NULL,
    Reviews VARCHAR(100000) NOT NULL,
    Feature VARCHAR(64) NOT NULL,
    Positive_rate FLOAT(24),
    Negative_rate FLOAT(24),
    Neutral_rate FLOAT(24),
    PRIMARY KEY (Reviews, Feature)
    CONSTRAINT FK_Reviews
    FOREIGN KEY (Reviews)
    REFERENCES review_user(Reviews)
)
"""

try:
    connection.execute(statement2)
except sa.exc.SQLAlchemyError as err:
    print("CREATE of review_feature failed:", str(err))

In [38]:
review_feature.to_sql("review_feature", con=connection, if_exists="append", index=False)

81

In [39]:
df2 = pd.read_sql_table("review_feature", connection)
df2

,Product,Reviews,Feature,Positive_rate,Negative_rate,Neutral_rate
0,Dyson,Background: \nThis is my first dyson. I am com...,Suction,0.242,0.047,0.711
1,Dyson,Background: \nThis is my first dyson. I am com...,Price,0.242,0.047,0.711
2,Dyson,It is an expensive vacuum we got a discount on...,Price,0.106,0.110,0.784
3,Dyson,"I do laundry, dishes, you name the chore, I do...",Suction,0.164,0.071,0.766
4,Dyson,First and only battery powered vacuum I’ve eve...,Suction,0.143,0.000,0.857
...,...,...,...,...,...,...
76,Hoover,I bought this for a small apartment. I use it...,Suction,0.155,0.017,0.828
77,Hoover,"Very pleased with the vacuum, It replaced my ...",Price,0.386,0.053,0.561
78,Hoover,I was tired of renting vacuum. I need it all i...,Suction,0.000,0.121,0.879
79,Hoover,You can never go wrong with a Hoover! Our old ...,Suction,0.248,0.070,0.682


**Create `review_rating` table**

In [40]:
statement3 = """
    CREATE TABLE IF NOT EXISTS review_rating(
    UserID VARCHAR(100) NOT NULL,
    Reviews VARCHAR(100000) NOT NULL,
    Ratings INT,
    PRIMARY KEY(Reviews, UserID)
    CONSTRAINT FK_Reviews
    FOREIGN KEY (Reviews, UserID)
    REFERENCES review_user(Reviews, UserID)
)
"""
try:
    connection.execute(statement3)
except sa.exc.SQLAlchemyError as err:
    print("CREATE of review_rating failed:", str(err))

In [41]:
review_rating.to_sql("review_rating", con=connection, if_exists="append", index=False)

80

In [42]:
df5 = pd.read_sql_table("review_rating", connection)
df5

,UserID,Reviews,Ratings
0,btd42,Background: \nThis is my first dyson. I am com...,5
1,AGK2000,It is an expensive vacuum we got a discount on...,5
2,LonG,"I do laundry, dishes, you name the chore, I do...",5
3,bestbuy,First and only battery powered vacuum I’ve eve...,5
4,LPerera,Simply put this vacume is built so well and ca...,5
...,...,...,...
75,HenryR,Seems to work just fine have not used all the ...,4
76,NJGecko,I bought this for a small apartment. I use it...,5
77,Smokie,"Very pleased with the vacuum, It replaced my ...",5
78,Anonymous,I was tired of renting vacuum. I need it all i...,5


In [43]:
review_rating.head(10)

,UserID,Reviews,Ratings
0,btd42,Background: \nThis is my first dyson. I am com...,5
1,AGK2000,It is an expensive vacuum we got a discount on...,5
2,LonG,"I do laundry, dishes, you name the chore, I do...",5
3,bestbuy,First and only battery powered vacuum I’ve eve...,5
4,LPerera,Simply put this vacume is built so well and ca...,5
5,Goose,"Best vacuum every, very strong and db levels a...",5
6,Karim,The best cordless dyson vacuum yet! From the s...,5
7,Roxie,I love this vacuum but I will say it is heavie...,5
8,User,Great for picking up dust an animal here. ￼The...,5
9,Juanber44,Just what i was expecting when i saw it for fi...,5


In [44]:
#Close the connection
connection.close()